# 📘 Notebook 1: Feature Engineering & ML Datasets

## Section 1 — Introduction

### Business Case: Wafer Yield Forecasting

Semiconductor manufacturing is one of the most complex industrial processes, with **yield** being the ultimate measure of success. A single percentage point improvement in yield can translate to millions of dollars in savings for a fab.

**Why Predict Yield?**
- **Early Warning System**: Identify at-risk wafers before final test
- **Process Optimization**: Correlate parameters with yield outcomes
- **Equipment Maintenance**: Detect drift before it impacts production
- **Cost Reduction**: Reduce scrap, rework, and warranty costs

### What's Covered

| Section | Topic |
|---------|-------|
| 2 | Load Data with Snowpark DataFrames |
| 3 | Feature Engineering Pipelines |
| 4 | Feature Store Registration |
| 5 | ML Dataset Creation (handoff to Notebook 2) |
| 6 | Best Practices |

### Dataset Overview (from Notebook 0)

| Table | Description | Key Features |
|-------|-------------|--------------|
| `WAFER_PROCESS_DATA` | Process telemetry (temp, pressure, gas flow) | Time-series profiles per step |
| `WAFER_DEFECT_LOGS` | Defect inspection results | Count, type, severity |
| `FINAL_YIELD_LABELS` | Binary yield outcomes | Target variable + root cause |

### References

- [Snowpark Python DataFrames](https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes)
- [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowpark-ml/feature-store/overview)
- [Snowflake ML Datasets](https://docs.snowflake.com/en/developer-guide/snowpark-ml/dataset)


In [ ]:
# ============================================================================
# IMPORTS & SESSION SETUP
# ============================================================================

import numpy as np
import pandas as pd
from datetime import datetime
from typing import List, Dict

# Snowpark imports
from snowflake.snowpark import Session
from snowflake.snowpark.functions import (
    col, lit, avg, sum as sum_, max as max_, min as min_, 
    stddev, count, when, coalesce, array_size, get,
    datediff, dayofweek, hour, to_variant, parse_json,
    flatten, lag, lead, row_number
)
from snowflake.snowpark.types import (
    StructType, StructField, StringType, IntegerType, 
    FloatType, TimestampType, VariantType
)
from snowflake.snowpark.window import Window

# Snowflake ML Feature Store
from snowflake.ml.feature_store import (
    FeatureStore, 
    FeatureView, 
    Entity,
    CreationMode
)

# Snowflake ML Dataset (for creating training data handoff)
from snowflake.ml import dataset

print("✅ Imports complete")


In [ ]:
# ============================================================================
# SNOWFLAKE SESSION (Snowflake Notebook)
# ============================================================================

from snowflake.snowpark.context import get_active_session

# Get the active session (automatically available in Snowflake Notebooks)
session = get_active_session()

# Set context
session.sql("USE DATABASE WAFER_YIELD_DEMO").collect()
session.sql("USE SCHEMA RAW_DATA").collect()

print(f"✅ Session active")
print(f"   Database: {session.get_current_database()}")
print(f"   Schema: {session.get_current_schema()}")


---

## 📘 Section 2 — Load Data with Snowpark DataFrames

We use **Snowpark DataFrames** with lazy evaluation for feature engineering. Data stays in Snowflake until an action triggers computation.

### Why Snowpark DataFrames for Feature Engineering?

| Benefit | Description |
|---------|-------------|
| **Lazy Evaluation** | Transformations are optimized before execution |
| **Pushdown** | All computation runs in Snowflake's engine |
| **SQL + Python** | Seamlessly combine SQL and DataFrame APIs |
| **Scalability** | Handles large datasets without memory issues |

> **Note**: For ML training data loading, Notebook 2 uses **Data Connector** for efficient pandas/PyTorch conversion.

In [ ]:
# ============================================================================
# LOAD SOURCE TABLES AS SNOWPARK DATAFRAMES
# ============================================================================

# Load tables as Snowpark DataFrames - no data is fetched yet (lazy evaluation)
process_df = session.table("RAW_DATA.WAFER_PROCESS_DATA")
defect_df = session.table("RAW_DATA.WAFER_DEFECT_LOGS")
yield_df = session.table("RAW_DATA.FINAL_YIELD_LABELS")

print("📋 Snowpark DataFrames loaded (lazy evaluation - no data transferred yet)")
print(f"   WAFER_PROCESS_DATA: {len(process_df.columns)} columns")
print(f"   WAFER_DEFECT_LOGS: {len(defect_df.columns)} columns")
print(f"   FINAL_YIELD_LABELS: {len(yield_df.columns)} columns")


In [ ]:
# ============================================================================
# DATA EXPLORATION (triggers query execution)
# ============================================================================

print("📊 Table Row Counts:")
print(f"   WAFER_PROCESS_DATA: {process_df.count():,} rows")
print(f"   WAFER_DEFECT_LOGS: {defect_df.count():,} rows")
print(f"   FINAL_YIELD_LABELS: {yield_df.count():,} rows")

# Preview data (limit reduces data transfer)
print("\n🔍 Sample Yield Labels:")
yield_df.limit(5).show()


---

## 📘 Section 3 — Feature Engineering Pipelines

We create production-ready features using **Snowpark DataFrame transformations**. All computations run in Snowflake's compute layer with automatic pushdown optimization.

### Feature Categories

| Category | Features | Description |
|----------|----------|-------------|
| **Temperature** | Max, min, mean, std, range | Extracted from temperature profiles |
| **Pressure** | Max, min, mean, std, delta | Extracted from pressure profiles |
| **Process** | Gas flow, humidity stats | Process parameter aggregations |
| **Defect** | Count, severity, density | Defect inspection metrics |
| **Temporal** | Hour, day-of-week | Time-based patterns |

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: TEMPERATURE FEATURES
# ============================================================================
# Snowpark DataFrame API - all operations are LAZY until an action is called

from snowflake.snowpark.functions import (
    col, avg, max as max_, min as min_, stddev, count, parse_json, lit
)

# ──────────────────────────────────────────────────────────────────────────────
# LAZY OPERATIONS (No execution yet - just building the query plan)
# ──────────────────────────────────────────────────────────────────────────────

# Flatten JSON arrays using table function
temp_flattened = (
    process_df
    .select("WAFER_ID", "TEMPERATURE_PROFILE")                              # Lazy
    .join_table_function("flatten", parse_json(col("TEMPERATURE_PROFILE"))) # Lazy
    .select(col("WAFER_ID"), col("VALUE").cast("FLOAT").alias("TEMP"))      # Lazy
)
# At this point: NO DATA HAS BEEN PROCESSED - only a query plan exists

# Aggregate temperature statistics per wafer
temp_features_df = (
    temp_flattened
    .group_by("WAFER_ID")  # Lazy
    .agg(                   # Lazy
        max_("TEMP").alias("TEMP_MAX"),
        min_("TEMP").alias("TEMP_MIN"),
        avg("TEMP").alias("TEMP_MEAN"),
        stddev("TEMP").alias("TEMP_STD"),
        (max_("TEMP") - min_("TEMP")).alias("TEMP_RANGE")
    )
)
# Still no execution - temp_features_df is just a query plan

print("✅ Temperature features DataFrame created (lazy - not yet executed)")

# ──────────────────────────────────────────────────────────────────────────────
# ACTION (Triggers execution of the entire query plan)
# ──────────────────────────────────────────────────────────────────────────────
temp_features_df.limit(5).show()  # NOW the query runs in Snowflake!


In [ ]:
# ============================================================================
# FEATURE ENGINEERING: PRESSURE FEATURES
# ============================================================================
# All operations below are LAZY - building query plan only

# Flatten and aggregate (LAZY - no execution)
pressure_flattened = (
    process_df
    .select("WAFER_ID", "PRESSURE_PROFILE")                              # Lazy
    .join_table_function("flatten", parse_json(col("PRESSURE_PROFILE"))) # Lazy
    .select(col("WAFER_ID"), col("VALUE").cast("FLOAT").alias("PRESSURE")) # Lazy
)

pressure_features_df = (
    pressure_flattened
    .group_by("WAFER_ID")  # Lazy
    .agg(                   # Lazy
        max_("PRESSURE").alias("PRESSURE_MAX"),
        min_("PRESSURE").alias("PRESSURE_MIN"),
        avg("PRESSURE").alias("PRESSURE_MEAN"),
        stddev("PRESSURE").alias("PRESSURE_STD"),
        (max_("PRESSURE") - min_("PRESSURE")).alias("PRESSURE_DELTA_MAX")
    )
)

print("✅ Pressure features DataFrame created (lazy)")
pressure_features_df.limit(5).show()  # ACTION - executes query


In [ ]:
# ============================================================================
# FEATURE ENGINEERING: PROCESS FEATURES
# ============================================================================
# Aggregate gas flow, humidity, and process step metrics per wafer

process_features_df = (
    process_df
    .group_by("WAFER_ID")
    .agg(
        # Gas flow statistics
        avg("GAS_FLOW_RATE").alias("GAS_FLOW_MEAN"),
        stddev("GAS_FLOW_RATE").alias("GAS_FLOW_STD"),
        max_("GAS_FLOW_RATE").alias("GAS_FLOW_MAX"),
        min_("GAS_FLOW_RATE").alias("GAS_FLOW_MIN"),
        
        # Humidity statistics
        avg("AMBIENT_HUMIDITY").alias("HUMIDITY_MEAN"),
        stddev("AMBIENT_HUMIDITY").alias("HUMIDITY_STD"),
        max_("AMBIENT_HUMIDITY").alias("HUMIDITY_MAX"),
        
        # Process metadata
        count("PROCESS_STEP").alias("NUM_PROCESS_STEPS"),
        count("EQUIPMENT_ID").alias("NUM_EQUIPMENT_USED")
    )
)

print("✅ Process features created")
process_features_df.limit(5).show()


In [ ]:
# ============================================================================
# FEATURE ENGINEERING: DEFECT FEATURES
# ============================================================================

defect_features_df = (
    defect_df
    .group_by("WAFER_ID")
    .agg(
        # Defect count metrics
        sum_("DEFECT_COUNT").alias("TOTAL_DEFECT_COUNT"),
        avg("DEFECT_COUNT").alias("AVG_DEFECT_COUNT"),
        max_("DEFECT_COUNT").alias("MAX_DEFECT_COUNT"),
        count("*").alias("NUM_INSPECTIONS"),
        
        # Severity metrics
        max_("SEVERITY_SCORE").alias("MAX_SEVERITY"),
        avg("SEVERITY_SCORE").alias("AVG_SEVERITY"),
        
        # Critical defect count (severity > 7)
        sum_(when(col("SEVERITY_SCORE") > 7, 1).otherwise(0)).alias("CRITICAL_DEFECT_COUNT")
    )
    # Calculate defect density
    .with_column(
        "DEFECT_DENSITY",
        col("TOTAL_DEFECT_COUNT") / col("NUM_INSPECTIONS")
    )
)

print("✅ Defect features created")
defect_features_df.limit(5).show()


In [ ]:
# ============================================================================
# FEATURE ENGINEERING: DOMINANT DEFECT TYPE
# ============================================================================
# Find the most common defect type per wafer using window functions

from snowflake.snowpark.functions import sum as sum_, row_number
from snowflake.snowpark.window import Window

# Count defects by type per wafer
defect_type_counts = (
    defect_df
    .group_by("WAFER_ID", "DEFECT_TYPE")
    .agg(sum_("DEFECT_COUNT").alias("TYPE_COUNT"))
)

# Rank defect types within each wafer
window_spec = Window.partition_by("WAFER_ID").order_by(col("TYPE_COUNT").desc())

dominant_defect_df = (
    defect_type_counts
    .with_column("RANK", row_number().over(window_spec))
    .filter(col("RANK") == 1)
    .select("WAFER_ID", col("DEFECT_TYPE").alias("DOMINANT_DEFECT_TYPE"))
)

print("✅ Dominant defect type features created")
dominant_defect_df.limit(5).show()

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: TEMPORAL FEATURES
# ============================================================================
# Extract time-based patterns using Snowpark DataFrame API

from snowflake.snowpark.functions import hour, dayofweek, datediff

# Get min/max timestamps per wafer and extract time features
temporal_features_df = (
    process_df
    .group_by("WAFER_ID")
    .agg(
        min_("TIMESTAMP").alias("FIRST_TIMESTAMP"),
        max_("TIMESTAMP").alias("LAST_TIMESTAMP")
    )
    .with_column("PROCESS_HOUR", hour(col("FIRST_TIMESTAMP")))
    .with_column("PROCESS_DAY_OF_WEEK", dayofweek(col("FIRST_TIMESTAMP")))
    .select("WAFER_ID", "PROCESS_HOUR", "PROCESS_DAY_OF_WEEK")
)

print("✅ Temporal features created")
temporal_features_df.limit(5).show()



In [ ]:
# ============================================================================
# JOIN ALL FEATURES INTO UNIFIED SNOWPARK DATAFRAME
# ============================================================================
# 
# KEY INSIGHT: All the DataFrames we created (temp_features_df, pressure_features_df, 
# process_features_df, etc.) are LAZY - they're just query plans, not materialized data.
#
# When we join them together, Snowflake's optimizer combines ALL the query plans
# into a SINGLE optimized execution plan. This is the power of lazy evaluation!
#
# ──────────────────────────────────────────────────────────────────────────────

# Combine all lazy DataFrames (still LAZY - just building a bigger query plan)
features_df = (
    yield_df
    .select("WAFER_ID", "YIELD_GOOD", "YIELD_SCORE")  # Lazy
    
    .join(temp_features_df, "WAFER_ID", "left")       # Lazy - incorporates temp query plan
    .join(pressure_features_df, "WAFER_ID", "left")   # Lazy - incorporates pressure query plan
    .join(process_features_df, "WAFER_ID", "left")    # Lazy - incorporates process query plan
    .join(defect_features_df, "WAFER_ID", "left")     # Lazy - incorporates defect query plan
    .join(dominant_defect_df, "WAFER_ID", "left")     # Lazy
    .join(temporal_features_df, "WAFER_ID", "left")   # Lazy
    
    .na.fill({                                         # Lazy
        "TOTAL_DEFECT_COUNT": 0,
        "AVG_DEFECT_COUNT": 0,
        "MAX_SEVERITY": 0,
        "AVG_SEVERITY": 0,
        "CRITICAL_DEFECT_COUNT": 0,
        "DEFECT_DENSITY": 0
    })
)

print("✅ All feature DataFrames joined (still lazy - no execution yet)")
print(f"   Total columns in query plan: {len(features_df.columns)}")

# ──────────────────────────────────────────────────────────────────────────────
# ACTION - This triggers execution of the ENTIRE combined query plan!
# Snowflake optimizes and executes all joins + aggregations in ONE query
# ──────────────────────────────────────────────────────────────────────────────
print(f"\n🚀 Executing combined query plan...")
print(f"   Total rows: {features_df.count():,}")  # ACTION - triggers execution
features_df.limit(5).show()                        # ACTION - shows results


In [ ]:
# ============================================================================
# FEATURE STATISTICS (computed on Snowpark DataFrame)
# ============================================================================

# Compute statistics without materializing to a table
stats_df = features_df.select(
    count("*").alias("TOTAL_WAFERS"),
    avg("YIELD_SCORE").alias("AVG_YIELD_SCORE"),
    sum_("YIELD_GOOD").alias("GOOD_WAFERS"),
    avg("TOTAL_DEFECT_COUNT").alias("AVG_DEFECTS"),
    avg("TEMP_MAX").alias("AVG_TEMP_MAX")
)

print("📊 Feature Statistics:")
stats_df.show()


---

## 📘 Section 4 — Register Feature Pipelines in Snowflake Feature Store

The **Snowflake Feature Store** provides:
- **Centralized feature repository** with metadata
- **Feature versioning** and lineage tracking
- **Point-in-time correct** feature retrieval
- **Feature reuse** across models and teams

In [ ]:
# ============================================================================
# INITIALIZE FEATURE STORE
# ============================================================================

# Create Feature Store instance
# CreationMode.CREATE_IF_NOT_EXIST creates the feature store schema if it doesn't exist
fs = FeatureStore(
    session=session,
    database="WAFER_YIELD_DEMO",
    name="FEATURES",
    default_warehouse="WAFER_DEMO_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

print("✅ Feature Store initialized")
print(f"   Database: WAFER_YIELD_DEMO")
print(f"   Schema: FEATURES")




In [ ]:
# ============================================================================
# CREATE ENTITY
# ============================================================================
# An Entity represents the primary key/join key for features

wafer_entity = Entity(
    name="WAFER_ENTITY",
    join_keys=["WAFER_ID"],
    desc="Wafer identifier for semiconductor manufacturing"
)

# Register entity (use FAIL_IF_NOT_EXIST for safety, CREATE_IF_NOT_EXIST for convenience)
fs.register_entity(wafer_entity)

print("✅ Wafer entity registered")

In [ ]:
# ============================================================================
# CREATE FEATURE VIEW
# ============================================================================

# Select features (excluding target and metadata columns) from the features_df
feature_columns = [
    "WAFER_ID",
    # Temperature features
    "TEMP_MAX", "TEMP_MIN", "TEMP_MEAN", "TEMP_STD", "TEMP_RANGE",
    # Pressure features  
    "PRESSURE_MAX", "PRESSURE_MIN", "PRESSURE_MEAN", "PRESSURE_STD", "PRESSURE_DELTA_MAX",
    # Process features
    "GAS_FLOW_MEAN", "GAS_FLOW_STD", "HUMIDITY_MEAN", "HUMIDITY_STD",
    # Defect features
    "TOTAL_DEFECT_COUNT", "AVG_DEFECT_COUNT", "MAX_SEVERITY", "AVG_SEVERITY",
    "CRITICAL_DEFECT_COUNT", "DEFECT_DENSITY",
    # Temporal features
    "PROCESS_HOUR", "PROCESS_DAY_OF_WEEK"
]

# Filter to available columns from our Snowpark DataFrame
available_cols = [c for c in feature_columns if c in features_df.columns]
feature_source_df = features_df.select(available_cols)

# Create Feature View
wafer_fv = FeatureView(
    name="WAFER_YIELD_FEATURES",
    entities=[wafer_entity],
    feature_df=feature_source_df,
    desc="Wafer yield prediction features including process, defect, and temporal metrics"
)

# Register with versioning
registered_fv = fs.register_feature_view(
    feature_view=wafer_fv,
    version="1.0",
    block=True  # Wait for registration to complete
)

print("✅ Feature View registered")
print(f"   Name: {registered_fv.name}")
print(f"   Version: {registered_fv.version}")

In [ ]:
# ============================================================================
# FEATURE METADATA (for documentation)
# ============================================================================

# Feature definitions for lineage tracking
feature_metadata = [
    ("TEMP_MAX", "Maximum temperature across all process steps"),
    ("TEMP_MEAN", "Average temperature across all readings"),
    ("TEMP_STD", "Temperature volatility measure"),
    ("PRESSURE_DELTA_MAX", "Maximum pressure variation"),
    ("TOTAL_DEFECT_COUNT", "Total defects found on wafer"),
    ("MAX_SEVERITY", "Highest defect severity score"),
    ("CRITICAL_DEFECT_COUNT", "Number of critical defects (severity > 7)"),
    ("PROCESS_HOUR", "Hour when processing started"),
]

print("📋 Feature Definitions:")
for feature_name, description in feature_metadata:
    print(f"   - {feature_name}: {description}")



In [ ]:
# ============================================================================
# RETRIEVE TRAINING DATA FROM FEATURE STORE
# ============================================================================

# Get the registered feature view
fv = fs.get_feature_view("WAFER_YIELD_FEATURES", "1.0")

# Get training data with labels
# Join with yield labels
labels_df = session.table("RAW_DATA.FINAL_YIELD_LABELS").select("WAFER_ID", "YIELD_GOOD", "YIELD_SCORE")

training_data = fv.feature_df.join(labels_df, "WAFER_ID")

print(f"✅ Training data retrieved: {training_data.count():,} rows")
print(f"   Features: {len(training_data.columns) - 3}")  # Exclude WAFER_ID, YIELD_GOOD, YIELD_SCORE
training_data.limit(5).show()

# Store reference for Dataset creation
TRAINING_DATA_DF = training_data



---

## 📘 Section 5 — ML Dataset Creation (Handoff to Notebook 2)

Create a **versioned ML Dataset** as the handoff artifact to Notebook 2 for model training.

### Why Create an ML Dataset?

| Benefit | Description |
|---------|-------------|
| **Immutable** | Training data is frozen for reproducibility |
| **Versioned** | Track which data was used for each experiment |
| **Efficient** | Notebook 2 loads via Data Connector |
| **Decoupled** | Data engineering ↔ Data science separation |


In [ ]:
# ============================================================================
# CREATE ML DATASET
# ============================================================================
# Reference: https://docs.snowflake.com/en/developer-guide/snowflake-ml/dataset

# Create a versioned dataset from our training data
# This creates an immutable snapshot for reproducible ML experiments

dataset_name = "WAFER_YIELD_TRAINING_DATASET"
dataset_version = "v2"

# Create dataset using create_from_dataframe
# This materializes the DataFrame into a Dataset version
wafer_dataset = dataset.create_from_dataframe(
    session,
    dataset_name,
    dataset_version,
    input_dataframe=TRAINING_DATA_DF
)

print(f"✅ ML Dataset created: {dataset_name}")
print(f"   Version: {dataset_version}")
print(f"   Rows: {wafer_dataset.read.to_snowpark_dataframe().count():,}")


In [ ]:
# ============================================================================
# VERIFY DATASET CREATION
# ============================================================================

# List versions in this dataset
print(f"📦 Dataset: {dataset_name}")
print(f"   Versions: {wafer_dataset.list_versions()}")
print(f"   Selected version: {wafer_dataset.selected_version.name}")

# Verify the dataset can be read
print(f"\n✅ Dataset ready for Notebook 2")
print(f"   Load command: dataset.load_dataset(session, '{dataset_name}', '{dataset_version}')")


In [ ]:
# ============================================================================
# DATASET SUMMARY
# ============================================================================

# Quick stats from the Snowpark DataFrame (before dataset creation)
print(f"📊 Training Dataset Summary:")
print(f"   Total rows: {TRAINING_DATA_DF.count():,}")
print(f"   Total columns: {len(TRAINING_DATA_DF.columns)}")
print(f"\n📋 Columns included in ML Dataset:")
for i, col in enumerate(TRAINING_DATA_DF.columns):
    print(f"   {i+1}. {col}")


---

## 📘 Section 6 — Best Practices

### Pipeline Flow

```
Notebook 1                                              Notebook 2
─────────────────────────────────────────────────────────────────────────
Raw Tables → Snowpark Transformations → Feature Store → ML Dataset → Data Connector → Training
                                                              │
                                                        (handoff point)
```

### When to Use Each Component

| Component | Where Used | Best For |
|-----------|------------|----------|
| **Snowpark DataFrame** | Notebook 1 | Feature engineering, transformations |
| **Feature Store** | Notebook 1 | Feature reuse across models |
| **ML Dataset** | Notebook 1 → 2 | Versioned training data handoff |
| **Data Connector** | Notebook 2 | Efficient pandas/PyTorch loading |

### Reproducibility Best Practices

1. **Version Everything**: Feature views, datasets, and models
2. **Use ML Datasets**: Immutable snapshots for experiments
3. **Document Transformations**: Store logic in feature metadata
4. **Use Point-in-Time Joins**: Prevent data leakage


In [ ]:
# ============================================================================
# NOTEBOOK SUMMARY
# ============================================================================

print("="*60)
print("🎉 NOTEBOOK 1 COMPLETE")
print("="*60)

print("\n📊 Features Created:")
print(f"   - Temperature features: 5 (max, min, mean, std, range)")
print(f"   - Pressure features: 5 (max, min, mean, std, delta)")
print(f"   - Process features: 4 (gas flow, humidity)")
print(f"   - Defect features: 6 (count, severity, density)")
print(f"   - Temporal features: 2 (hour, day-of-week)")
print(f"   - Total: ~22 features")

print("\n📦 Artifacts Created:")
print(f"   - Feature Table: FEATURES.WAFER_FEATURES_MASTER")
print(f"   - Feature View: WAFER_YIELD_FEATURES v1.0")
print(f"   - ML Dataset: {dataset_name} {dataset_version}")
print(f"   - Entity: WAFER_ENTITY")

print("\n➡️ Next: Notebook 2")
print("   Loads ML Dataset via Data Connector for model training")


In [ ]:
-- List all datasets in the database
SHOW DATASETS IN DATABASE WAFER_YIELD_DEMO;

-- Or in a specific schema
SHOW DATASETS IN SCHEMA WAFER_YIELD_DEMO.RAW_DATA;